# 🚀 NovaGen AI - Full Stack Server (Fastify + Python)
Este notebook despliega el entorno completo de NovaGen AI en Google Colab.

- ⚡ **Node.js Gateway**: Servidor Fastify para gestión de API y WebSockets (Puerto 3000).
- 🧠 **Python AI Engine**: Motor de inferencia Juggernaut-XL y LLM (Puerto 7860).
- 🌐 **Acceso Remoto**: Túnel Ngrok único apuntando al gateway de Node.js.
- 📥 **Clonado Automático**: Sincronización con el repositorio oficial.
- ⚡ **GPU T4**: Optimización de VRAM y xformers.

In [ ]:
# @title 1. Inicialización del Entorno (Clone & Install)
import os
import subprocess
import sys

print("🚀 Iniciando secuencia de arranque...")

# 1. Verificar GPU
import torch
if torch.cuda.is_available():
    print(f"✅ GPU Detectada: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ ADVERTENCIA: GPU no detectada. Ve a 'Entorno de ejecución' > 'Cambiar tipo' > 'T4 GPU'.")

# 2. Clonar/Actualizar Repositorio
REPO_URL = "https://github.com/Juanpalojime/NovaGenIA.git"
REPO_NAME = "NovaGenIA"

if os.path.exists(REPO_NAME):
    print("🔄 Actualizando repositorio...")
    %cd {REPO_NAME}
    !git pull
    %cd ..
else:
    print("⬇ Clonando repositorio...")
    !git clone {REPO_URL}

# 3. Instalar Dependencias de Python
print("⏳ Instalando dependencias de Python...")
!pip install -r {REPO_NAME}/requirements.txt -q
!pip install pyngrok nest_asyncio huggingface_hub websockets -q

# 4. Instalar Node.js y dependencias del Gateway
print("📦 Configurando Node.js y Fastify Gateway...")
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - > /dev/null
!sudo apt-get install -y nodejs > /dev/null

if os.path.exists(f"{REPO_NAME}/server-node"):
    %cd {REPO_NAME}/server-node
    !npm install > /dev/null
    %cd ../..
    print("✅ Node.js Gateway preparado.")
else:
    print("⚠️ No se encontró la carpeta server-node.")

print("✅ Entorno preparado.")

In [ ]:
# @title 1.5 Mount Google Drive (Optional)
from google.colab import drive
import os

print("📂 Mounting Google Drive...")
try:
    drive.mount('/content/drive')
    print("✅ Drive mounted at /content/drive")
    drive_output_path = "/content/drive/MyDrive/NovaGen_Outputs"
    os.makedirs(drive_output_path, exist_ok=True)
except Exception as e:
    print(f"⚠️ Drive mount skipped or failed: {e}")

In [ ]:
# @title 2. Configuración de Credenciales (Ngrok & HF)
from pyngrok import ngrok
import os

# @markdown ### Ngrok Setup
NGROK_AUTH_TOKEN = "" # @param {type:"string"}

# @markdown ### HuggingFace Setup
HF_TOKEN = "" # @param {type:"string"}

if NGROK_AUTH_TOKEN:
    clean_token = NGROK_AUTH_TOKEN.replace("ngrok config add-authtoken ", "").strip()
    ngrok.set_auth_token(clean_token)
    print(f"✅ Ngrok configurado.")

os.environ["HF_TOKEN"] = HF_TOKEN
if HF_TOKEN:
    !huggingface-cli login --token {HF_TOKEN}

In [ ]:
# @title 3. Descarga de Modelos
import os

MODELS_DIR = f"{REPO_NAME}/models/checkpoints"
LORAS_DIR = f"{REPO_NAME}/models/loras"
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LORAS_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODELS_DIR, "Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors")
if not os.path.exists(MODEL_PATH):
    URL = "https://civitai.com/api/download/models/456194"
    print(f"⬇ Descargando modelo base (~6.5 GB)...")
    !wget -O {MODEL_PATH} "{URL}" --progress=bar:force:noscroll

# FaceSwap
FACESWAP_DIR = f"{REPO_NAME}/models/insightface"
os.makedirs(FACESWAP_DIR, exist_ok=True)
SWAP_MODEL_PATH = os.path.join(FACESWAP_DIR, "inswapper_128.onnx")
if not os.path.exists(SWAP_MODEL_PATH):
    !wget -O {SWAP_MODEL_PATH} "https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx"

print("✅ Modelos listos.")

In [ ]:
# @title 4. INICIAR SISTEMA (Dual Backend)
import nest_asyncio
import time
import subprocess
from pyngrok import ngrok

nest_asyncio.apply()

print("🔄 Reiniciando servicios...")
!pkill -f uvicorn
!pkill -f node
ngrok.kill()
time.sleep(2)

# Iniciar Túnel Ngrok al Gateway de Node.js (Puerto 3000)
PORT_GATEWAY = 3000
try:
    tunnel = ngrok.connect(PORT_GATEWAY)
    print("="*50)
    print(f"🔗 PUBLIC URL: \033[96m{tunnel.public_url}\033[0m")
    print("="*50)
    print("👉 Copia esta URL en NovaGen > Settings > Connections")

    # 1. Iniciar Python Inerence Engine (Puerto 7860)
    print("⏳ Iniciando Motor Python (Inferencia)...")
    python_proc = subprocess.Popen(["python", "server.py"], cwd=REPO_NAME)
    
    time.sleep(5) # Esperar un poco a que Python inicie

    # 2. Iniciar Node.js Gateway (Puerto 3000)
    print("⏳ Iniciando Gateway Node.js (Fastify)...")
    node_proc = subprocess.Popen(["node", "index.js"], cwd=f"{REPO_NAME}/server-node")

    print("🚀 Sistema en línea. Los logs se mostrarán a continuación...")
    
    try:
        # Mantener el proceso vivo
        python_proc.wait()
    except KeyboardInterrupt:
        python_proc.terminate()
        node_proc.terminate()
        print("🛑 Servidores detenidos.")

except Exception as e:
    print(f"❌ Error al iniciar: {e}")